In [23]:
# Install necessary packages
!pip install -q MRdataset
!pip install -q mrQA

In [24]:
# Import required libraries
import IPython
from MRdataset import import_dataset
from mrQA import check_compliance

In [25]:
# Define data paths
data_root = '/home/jovyan/shared/ds003688'  # Change path here
output_dir = '/home/jovyan/group_project/mrqa_reports/'  # Change path here

In [26]:
# Import the dataset
dataset = import_dataset(data_source=data_root,
                         ds_format='bids',
                         name='ds003688')

In [27]:
# Print dataset information
print(dataset)

FastBIDSDataset ds003688 with 2 Modality


In [28]:
# Print modalities and subjects
for modality in dataset.modalities:
    print(modality)

Modality anat_T1w with 63 Subject
Modality func_task-film_bold with 30 Subject


In [29]:
# Check compliance and generate a report
check_compliance(dataset=dataset, output_dir=output_dir)

In ds003688 dataset, modalities "anat_T1w" are non-compliant. See /home/jovyan/group_project/mrqa_reports/ds003688_DATE_08_15_2023_05_14_58.html for report


PosixPath('/home/jovyan/group_project/mrqa_reports/ds003688_DATE_08_15_2023_05_14_58.html')

In [ ]:
import pandas as pd
import numpy as np

#Loops through every subject, session, and run and puts it in a dictionary structure 
#ex. {Modality:{'MagneticField Strength':3}, {'MagneticField Strength':3, EchoTime: 1} }
our_params = dict()
for modality in dataset.modalities:
    #print(modality)
    our_params[str(modality)] = []
    for subject in modality.subjects:
                for session in subject.sessions:
                    for run in session.runs:
                            #print(run.params)
                            our_params[str(modality)].append(run.params)
#print(our_params)        


# This puts our dictionary structure into a dataframe
test_df = pd.DataFrame.from_dict(our_params, orient='index').T
test_df = test_df.reset_index().rename(columns={'index': 'Run_number'})
#test_df


#Elongates table so that Modalities is just a value for each run and each run is represented in a row
modalities = [c for c in test_df if c.startswith('Modality')]
test_df=pd.melt(test_df, id_vars='Run_number', value_vars=modalities, value_name='Modality')
test_df = test_df.rename(columns={'Modality': 'Value', 'variable': 'Modality'}) #Fixes some of the names
#test_df
test_df=test_df[test_df["Value"].notnull()] #Rid of empty values 

#Works as examples
#test_df['MagneticFieldStrength'] = test_df["Value"].apply(lambda x: x.get("MagneticFieldStrength", None))

## Extraction of values for each specified parameter into their own columns
#Names of parmeters specified from github
PARAMETER_NAMES = [
    'Manufacturer',
    'BodyPartExamined',
    'RepetitionTime',
    'MagneticFieldStrength',
    'FlipAngle',
    'EchoTrainLength',
    'PixelBandwidth',
    'NumberOfPhaseEncodingSteps',
    ]

for param_name in PARAMETER_NAMES:
    test_df[param_name] = test_df["Value"].apply(lambda x: x.get(param_name, None))

#Names not included on github
EXTRA_PARAMETERS = [
    'EchoTime',
    'PhaseEncodingDirection',
    'ScanningSequence',
    'RepetitionTimeExcitation',
    'SequenceVariant',
    'MRAcquisitionType'
    
    ]
for param_name in EXTRA_PARAMETERS:
    test_df[param_name] = test_df["Value"].apply(lambda x: x.get(param_name, None))

#test_df
    
    

    